In [1]:
import CSV
using DataFrames
# using StatsBase
include("$(dirname(dirname(pwd())))/pred_funcs.jl")
using MLJ
# import MLJBase
import TableView.showtable
using Plots
import FileIO
import Statistics
import LinearAlgebra
using StatsPlots
using Random
Random.seed!(0);
using StatsBase
import MLJBase

WebIO._IJuliaInit()

In [2]:
MissingImputator = @load MissingImputator pkg="BetaML" verbosity=0
LogisticClassifier = @load LogisticClassifier pkg="MLJLinearModels" verbosity=0;  # loads code defining a model type
EvoTreeClassifier = @load EvoTreeClassifier verbosity=0

EvoTrees.EvoTreeClassifier

In [3]:
full = ""

""

In [4]:

sample_key2, xold, yold = FileIO.load("$(dirname(dirname(pwd())))/data$(full).jld2","sample_key","x" ,"y");


xold.col1=collect(1:nrow(xold));
println(nrow(xold))
# filter!(x->x.ALTREF_I ≈ 0,xold)
println(nrow(xold))
# filter!(x->x.ALT_I>0.5 || x.REF_I>0.5,xold) # indel
# filter!(x->x.ALT_I<0.5 && x.REF_I<0.5,xold) # SNV
println(nrow(xold))
filter!(x->x.training>0.5,xold)
# select!(xold,Not("training"));
println(nrow(xold))
# filter!(x->!ismissing(x.dust_score_Mutect),xold);
# println(nrow(xold))

yold = yold[xold.col1]
# println(xold[findall(x->x,[i==true for i in yold]),"col1"])
# println(xold[findall(x->!x,[i==true for i in yold]),"col1"])
sample_key2 = sample_key2[xold.col1]
select!(xold,Not("col1"));

94626
94626
94626
26217


In [5]:
for j in 1:ncol(xold), i in 1:nrow(xold)
    if !ismissing(xold[i,j]) && (isnan(xold[i,j]) || isinf(xold[i,j]) )
        xold[i,j] = missing
    end
end

In [6]:
train2, test2 = partition(eachindex(yold), 0.5, stratify=yold,  shuffle=true, rng=12344);

select!(xold,Not("training"));

In [7]:
sum([i==true for i in yold[train2]])

46

In [8]:
sum([i==true for i in yold[test2]])

46

In [9]:
sum(.~ismissing.(xold[train2,:dust_score_Vardict]))

8023

In [10]:
FileIO.save("testdata$(full).jld2", Dict("sample_key"=>sample_key2[test2],"x" => xold[test2,:],
        "y" => yold[test2], 
    ) ; compress = true
)
FileIO.save("traindata$(full).jld2", Dict("sample_key"=>sample_key2[train2],"x" => xold[train2,:],
        "y" => yold[train2], 
    ) ; compress = true
)

# Mutect

In [11]:
x = copy(xold[train2,:]);
y = copy(yold[train2]);

In [12]:
x.col1=collect(1:nrow(x));
coldrop = [] #"FPpass","FILTER_Mutect_PASS"] # ["gt_AF_Mutect","AltFwd_Mutect","RefFwd_Mutect","AltRev_Mutect","RefRev_Mutect","gt_AD_ref_Mutect","gt_AD_alt_Mutect"]
for i in names(x)
    if i == "col1" || i=="training" || occursin( "FILTER_Mutect",i)
        continue
    end
    if length(unique(x[:,i])) < 2 || occursin("Pindel",i) || occursin("Lofreq",i) || occursin("Vardict",i) # || occursin("ALT",i) || occursin("REF",i)
        push!(coldrop,i)
#         println(i)
    end 
end
unique!(coldrop)
select!(x, Not(coldrop));
println(sum([i==true for i in y]))
# println(nrow(xold))
# dropmissing!(xold)
# println(nrow(xold))
filter!(x->!ismissing(x.dust_score_Mutect),x);
println(size(y))
# println(length(sample_key2))
println()
y = y[x.col1]
# println(xold[findall(x->x,[i==true for i in yold]),"col1"])
# println(xold[findall(x->!x,[i==true for i in yold]),"col1"])
# sample_key2 = sample_key2[x.col1]
select!(x,Not("col1"));

println(size(x))
println(size(y))
# println(length(sample_key2))
println(sum([i==true for i in y]));


46
(13108,)

(817, 70)
(817,)
39


In [13]:
transform_log!(x,10)

In [14]:
stnd = fit_standardizer(x)
transform_standardizer!(x,stnd);

In [15]:
println(sum(ismissing.(Matrix(x))))

18


In [16]:
imputemach = machine(MissingImputator(K=5),x) 
@time fit!(imputemach,verbosity=1)

┌ Info: Training Machine{MissingImputator,…}.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/HZmTU/src/machines.jl:403


 17.646865 seconds (47.33 M allocations: 3.124 GiB, 5.07% gc time)


Machine{MissingImputator,…} trained 1 time; caches data
  args: 
    1:	Source @033 ⏎ `Table{Union{AbstractVector{Union{Missing, Continuous}}, AbstractVector{Continuous}}}`


In [17]:
tt = MLJ.transform(imputemach,x);
for i in 1:ncol(x)
    x[:,i] = tt[i]
end

In [18]:
disallowmissing!(x);

In [19]:
println(size(x)," ",sum([i==true for i in y] ) )
println(size(y))


(817, 70) 39
(817,)


In [20]:
# x,y,train,test,sample_key = sampler(x,y,train,test,sample_key,"under",0.8);
# x,y,train,test,sample_key = sampler(x,y,train,test,sample_key,"over",0.2);

In [21]:
y = coerce(copy(y),OrderedFactor);

In [22]:
println(size(x)," ",sum([i==true for i in y] ) )
println(size(y))

(817, 70) 39
(817,)


In [23]:

r = [
    range(EvoTreeClassifier(), :max_depth, lower=2,upper=8),
    range(EvoTreeClassifier(), :η, lower=0.0,upper=1.0),
    range(EvoTreeClassifier(), :γ, lower=0.0,upper=1.0),
    range(EvoTreeClassifier(), :λ, lower=0.0,upper=1.0),
    range(EvoTreeClassifier(), :α, lower=0.0,upper=1.0),
    range(EvoTreeClassifier(), :min_weight, lower=0.0,upper=1.0),
    range(EvoTreeClassifier(), :nrounds, lower=1,upper=10),
    range(EvoTreeClassifier(), :nbins, lower=4, upper=128),
    
    
#     range(LogisticClassifier(), :lambda, lower=0.0,upper=1.0),
#     range(LogisticClassifier(), :gamma, lower=0.0,upper=1.0),
]

8-element Vector{MLJBase.NumericRange{T, MLJBase.Bounded, Symbol} where T}:
 NumericRange(2 ≤ max_depth ≤ 8; origin=5.0, unit=3.0)
 NumericRange(0.0 ≤ η ≤ 1.0; origin=0.5, unit=0.5)
 NumericRange(0.0 ≤ γ ≤ 1.0; origin=0.5, unit=0.5)
 NumericRange(0.0 ≤ λ ≤ 1.0; origin=0.5, unit=0.5)
 NumericRange(0.0 ≤ α ≤ 1.0; origin=0.5, unit=0.5)
 NumericRange(0.0 ≤ min_weight ≤ 1.0; origin=0.5, unit=0.5)
 NumericRange(1 ≤ nrounds ≤ 10; origin=5.5, unit=4.5)
 NumericRange(4 ≤ nbins ≤ 128; origin=66.0, unit=62.0)

In [24]:
self_tune = TunedModel(
#     model=LogisticClassifier(penalty=:l1),
    model=EvoTreeClassifier(),
    
    resampling=StratifiedCV(nfolds=5, rng=123),

    tuning=LatinHypercube(),  #     tuning=Grid(resolution=2), 
    range = r,
    n=200,
    operation=predict_mode, # use for probabalistic 
    measure = fnr,  # use for deterministic or with predict_mode for probab 
    acceleration=CPUThreads(),
)

machbest = machine(
    self_tune,
    x,y)
fit!(machbest,verbosity=1 )



┌ Info: Training Machine{ProbabilisticTunedModel{LatinHypercube,…},…}.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/HZmTU/src/machines.jl:403
┌ Info: Attempting to evaluate 200 models.
└ @ MLJTuning /Users/alexpanchot/.julia/packages/MLJTuning/efiDR/src/tuned_models.jl:685
Evaluating over 200 metamodels: 100%[=========================] Time: 0:01:04


Machine{ProbabilisticTunedModel{LatinHypercube,…},…} trained 1 time; caches data
  args: 
    1:	Source @175 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @986 ⏎ `AbstractVector{OrderedFactor{2}}`


In [25]:
mach = machine(
    fitted_params(machbest).best_model,
    x,y)
fit!(mach,verbosity=4 )

┌ Info: Training Machine{EvoTreeClassifier{Float64,…},…}.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/HZmTU/src/machines.jl:403


Machine{EvoTreeClassifier{Float64,…},…} trained 1 time; caches data
  args: 
    1:	Source @727 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @614 ⏎ `AbstractVector{OrderedFactor{2}}`


In [26]:
FileIO.save("mutect$(full).jld2", Dict("mach"=>mach,"stnd" => stnd,
        "imputemach" => imputemach, 
    ) ; compress = true
)

# Vardict

In [27]:
x = copy(xold[train2,:]);
y = copy(yold[train2]);

In [28]:
x.col1=collect(1:nrow(x));
coldrop = [] #"FPpass","FILTER_Mutect_PASS"] # ["gt_AF_Mutect","AltFwd_Mutect","RefFwd_Mutect","AltRev_Mutect","RefRev_Mutect","gt_AD_ref_Mutect","gt_AD_alt_Mutect"]
for i in names(x)
    if i == "col1" || i=="training" || occursin( "FILTER_Vardict",i)
        continue
    end
    if length(unique(x[:,i])) < 2 || occursin("Pindel",i) || occursin("Lofreq",i) || occursin("Mutect",i) # || occursin("ALT",i) || occursin("REF",i)
        push!(coldrop,i)
#         println(i)
    end 
end
unique!(coldrop)
select!(x, Not(coldrop));
println(sum([i==true for i in y]))
# println(nrow(xold))
# dropmissing!(xold)
# println(nrow(xold))
filter!(x->!ismissing(x.dust_score_Vardict),x);
println(size(y))
# println(length(sample_key2))
println()
y = y[x.col1]
# println(xold[findall(x->x,[i==true for i in yold]),"col1"])
# println(xold[findall(x->!x,[i==true for i in yold]),"col1"])
# sample_key2 = sample_key2[x.col1]
select!(x,Not("col1"));

println(size(x))
println(size(y))
# println(length(sample_key2))
println(sum([i==true for i in y]));



46
(13108,)

(8023, 63)
(8023,)
46


In [29]:
transform_log!(x,10)
stnd = fit_standardizer(x)
transform_standardizer!(x,stnd);
println(sum(ismissing.(Matrix(x))))

1788


In [30]:
imputemach = machine(MissingImputator(K=5),x) 
@time fit!(imputemach,verbosity=1)

┌ Info: Training Machine{MissingImputator,…}.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/HZmTU/src/machines.jl:403


  5.779329 seconds (34.97 M allocations: 5.406 GiB, 17.77% gc time, 2.12% compilation time)


Machine{MissingImputator,…} trained 1 time; caches data
  args: 
    1:	Source @583 ⏎ `Table{Union{AbstractVector{Union{Missing, Continuous}}, AbstractVector{Continuous}}}`


In [31]:
tt = MLJ.transform(imputemach,x);
for i in 1:ncol(x)
    x[:,i] = tt[i]
end
disallowmissing!(x);

In [32]:
println(size(x)," ",sum([i==true for i in y] ) )
println(size(y))
# x,y,train,test,sample_key = sampler(x,y,train,test,sample_key,"under",0.8);
# x,y,train,test,sample_key = sampler(x,y,train,test,sample_key,"over",0.2);

y = coerce(copy(y),OrderedFactor);

println(size(x)," ",sum([i==true for i in y] ) )
println(size(y))

(8023, 63) 46
(8023,)
(8023, 63) 46
(8023,)


In [33]:

r = [
    range(EvoTreeClassifier(), :max_depth, lower=2,upper=8),
    range(EvoTreeClassifier(), :η, lower=0.0,upper=1.0),
    range(EvoTreeClassifier(), :γ, lower=0.0,upper=1.0),
    range(EvoTreeClassifier(), :λ, lower=0.0,upper=1.0),
    range(EvoTreeClassifier(), :α, lower=0.0,upper=1.0),
    range(EvoTreeClassifier(), :min_weight, lower=0.0,upper=1.0),
    range(EvoTreeClassifier(), :nrounds, lower=1,upper=10),
    range(EvoTreeClassifier(), :nbins, lower=4, upper=128),
    
    
#     range(LogisticClassifier(), :lambda, lower=0.0,upper=1.0),
#     range(LogisticClassifier(), :gamma, lower=0.0,upper=1.0),
]

8-element Vector{MLJBase.NumericRange{T, MLJBase.Bounded, Symbol} where T}:
 NumericRange(2 ≤ max_depth ≤ 8; origin=5.0, unit=3.0)
 NumericRange(0.0 ≤ η ≤ 1.0; origin=0.5, unit=0.5)
 NumericRange(0.0 ≤ γ ≤ 1.0; origin=0.5, unit=0.5)
 NumericRange(0.0 ≤ λ ≤ 1.0; origin=0.5, unit=0.5)
 NumericRange(0.0 ≤ α ≤ 1.0; origin=0.5, unit=0.5)
 NumericRange(0.0 ≤ min_weight ≤ 1.0; origin=0.5, unit=0.5)
 NumericRange(1 ≤ nrounds ≤ 10; origin=5.5, unit=4.5)
 NumericRange(4 ≤ nbins ≤ 128; origin=66.0, unit=62.0)

In [34]:
self_tune = TunedModel(
#     model=LogisticClassifier(penalty=:l1),
    model=EvoTreeClassifier(),
    
    resampling=StratifiedCV(nfolds=5, rng=123),

    tuning=LatinHypercube(),  #     tuning=Grid(resolution=2), 
    range = r,
    n=200,
    operation=predict_mode, # use for probabalistic 
    measure = fnr,  # use for deterministic or with predict_mode for probab 
    acceleration=CPUThreads(),
)

machbest = machine(
    self_tune,
    x,y)
fit!(machbest,verbosity=1 )

mach = machine(
    fitted_params(machbest).best_model,
    x,y)
fit!(mach,verbosity=1 )


┌ Info: Training Machine{ProbabilisticTunedModel{LatinHypercube,…},…}.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/HZmTU/src/machines.jl:403
┌ Info: Attempting to evaluate 200 models.
└ @ MLJTuning /Users/alexpanchot/.julia/packages/MLJTuning/efiDR/src/tuned_models.jl:685
Evaluating over 200 metamodels: 100%[=========================] Time: 0:00:44
┌ Info: Training Machine{EvoTreeClassifier{Float64,…},…}.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/HZmTU/src/machines.jl:403


Machine{EvoTreeClassifier{Float64,…},…} trained 1 time; caches data
  args: 
    1:	Source @548 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @995 ⏎ `AbstractVector{OrderedFactor{2}}`


In [35]:
FileIO.save("vardict$(full).jld2", Dict("mach"=>mach,"stnd" => stnd,
        "imputemach" => imputemach, 
    ) ; compress = true
)

# Lofreq

In [36]:
x = copy(xold[train2,:]);
y = copy(yold[train2]);

In [37]:
x.col1=collect(1:nrow(x));
coldrop = [] #"FPpass","FILTER_Mutect_PASS"] # ["gt_AF_Mutect","AltFwd_Mutect","RefFwd_Mutect","AltRev_Mutect","RefRev_Mutect","gt_AD_ref_Mutect","gt_AD_alt_Mutect"]
for i in names(x)
    if i == "col1" || i=="training" || occursin( "FILTER_Lofreq",i)
        continue
    end
    if length(unique(x[:,i])) < 2 || occursin("Pindel",i) || occursin("Vardict",i) || occursin("Mutect",i) # || occursin("ALT",i) || occursin("REF",i)
        push!(coldrop,i)
#         println(i)
    end 
end
unique!(coldrop)
select!(x, Not(coldrop));
println(sum([i==true for i in y]))
# println(nrow(xold))
# dropmissing!(xold)
# println(nrow(xold))
filter!(x->!ismissing(x.dust_score_Lofreq),x);
println(size(y))
# println(length(sample_key2))
println()
y = y[x.col1]
# println(xold[findall(x->x,[i==true for i in yold]),"col1"])
# println(xold[findall(x->!x,[i==true for i in yold]),"col1"])
# sample_key2 = sample_key2[x.col1]
select!(x,Not("col1"));

println(size(x))
println(size(y))
# println(length(sample_key2))
println(sum([i==true for i in y]));




46
(13108,)

(10016, 50)
(10016,)
46


In [38]:
transform_log!(x,10)
stnd = fit_standardizer(x)
transform_standardizer!(x,stnd);
println(sum(ismissing.(Matrix(x))))

1201


In [39]:
imputemach = machine(MissingImputator(K=5),x) 
@time fit!(imputemach,verbosity=1)

┌ Info: Training Machine{MissingImputator,…}.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/HZmTU/src/machines.jl:403


 13.443852 seconds (20.52 M allocations: 2.218 GiB, 3.82% gc time, 83.97% compilation time)


Machine{MissingImputator,…} trained 1 time; caches data
  args: 
    1:	Source @780 ⏎ `Table{Union{AbstractVector{Union{Missing, Continuous}}, AbstractVector{Continuous}}}`


In [40]:
tt = MLJ.transform(imputemach,x);
for i in 1:ncol(x)
    x[:,i] = tt[i]
end
disallowmissing!(x);

In [41]:
println(size(x)," ",sum([i==true for i in y] ) )
println(size(y))
# x,y,train,test,sample_key = sampler(x,y,train,test,sample_key,"under",0.8);
# x,y,train,test,sample_key = sampler(x,y,train,test,sample_key,"over",0.2);

y = coerce(copy(y),OrderedFactor);

println(size(x)," ",sum([i==true for i in y] ) )
println(size(y))

(10016, 50) 46
(10016,)
(10016, 50) 46
(10016,)


In [42]:

r = [
    range(EvoTreeClassifier(), :max_depth, lower=2,upper=8),
    range(EvoTreeClassifier(), :η, lower=0.0,upper=1.0),
    range(EvoTreeClassifier(), :γ, lower=0.0,upper=1.0),
    range(EvoTreeClassifier(), :λ, lower=0.0,upper=1.0),
    range(EvoTreeClassifier(), :α, lower=0.0,upper=1.0),
    range(EvoTreeClassifier(), :min_weight, lower=0.0,upper=1.0),
    range(EvoTreeClassifier(), :nrounds, lower=1,upper=10),
    range(EvoTreeClassifier(), :nbins, lower=4, upper=128),
    
    
#     range(LogisticClassifier(), :lambda, lower=0.0,upper=1.0),
#     range(LogisticClassifier(), :gamma, lower=0.0,upper=1.0),
]

8-element Vector{MLJBase.NumericRange{T, MLJBase.Bounded, Symbol} where T}:
 NumericRange(2 ≤ max_depth ≤ 8; origin=5.0, unit=3.0)
 NumericRange(0.0 ≤ η ≤ 1.0; origin=0.5, unit=0.5)
 NumericRange(0.0 ≤ γ ≤ 1.0; origin=0.5, unit=0.5)
 NumericRange(0.0 ≤ λ ≤ 1.0; origin=0.5, unit=0.5)
 NumericRange(0.0 ≤ α ≤ 1.0; origin=0.5, unit=0.5)
 NumericRange(0.0 ≤ min_weight ≤ 1.0; origin=0.5, unit=0.5)
 NumericRange(1 ≤ nrounds ≤ 10; origin=5.5, unit=4.5)
 NumericRange(4 ≤ nbins ≤ 128; origin=66.0, unit=62.0)

In [43]:
self_tune = TunedModel(
#     model=LogisticClassifier(penalty=:l1),
    model=EvoTreeClassifier(),
    
    resampling=StratifiedCV(nfolds=5, rng=123),

    tuning=LatinHypercube(),  #     tuning=Grid(resolution=2), 
    range = r,
    n=200,
    operation=predict_mode, # use for probabalistic 
    measure = fnr,  # use for deterministic or with predict_mode for probab 
    acceleration=CPUThreads(),
)

machbest = machine(
    self_tune,
    x,y)
fit!(machbest,verbosity=1 )

mach = machine(
    fitted_params(machbest).best_model,
    x,y)
fit!(mach,verbosity=1 )

┌ Info: Training Machine{ProbabilisticTunedModel{LatinHypercube,…},…}.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/HZmTU/src/machines.jl:403
┌ Info: Attempting to evaluate 200 models.
└ @ MLJTuning /Users/alexpanchot/.julia/packages/MLJTuning/efiDR/src/tuned_models.jl:685
Evaluating over 200 metamodels: 100%[=========================] Time: 0:00:35
┌ Info: Training Machine{EvoTreeClassifier{Float64,…},…}.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/HZmTU/src/machines.jl:403


Machine{EvoTreeClassifier{Float64,…},…} trained 1 time; caches data
  args: 
    1:	Source @228 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @458 ⏎ `AbstractVector{OrderedFactor{2}}`


In [44]:
FileIO.save("lofreq$(full).jld2", Dict("mach"=>mach,"stnd" => stnd,
        "imputemach" => imputemach, 
    ) ; compress = true
)

In [45]:
"lofreq$(full).jld2"

"lofreq.jld2"

# Pindel

In [46]:
x = copy(xold[train2,:]);
y = copy(yold[train2]);

In [47]:
x.col1=collect(1:nrow(x));
coldrop = [] #"FPpass","FILTER_Mutect_PASS"] # ["gt_AF_Mutect","AltFwd_Mutect","RefFwd_Mutect","AltRev_Mutect","RefRev_Mutect","gt_AD_ref_Mutect","gt_AD_alt_Mutect"]
for i in names(x)
    if i == "col1" || i=="training" || occursin( "FILTER_Pindel",i)
        continue
    end
    if length(unique(x[:,i])) < 2 || occursin("Lofreq",i) || occursin("Vardict",i) || occursin("Mutect",i) # || occursin("ALT",i) || occursin("REF",i)
        push!(coldrop,i)
#         println(i)
    end 
end
unique!(coldrop)
select!(x, Not(coldrop));
println(sum([i==true for i in y]))
# println(nrow(xold))
# dropmissing!(xold)
# println(nrow(xold))
filter!(x->!ismissing(x.dust_score_Pindel),x);
println(size(y))
# println(length(sample_key2))
println()
y = y[x.col1]
# println(xold[findall(x->x,[i==true for i in yold]),"col1"])
# println(xold[findall(x->!x,[i==true for i in yold]),"col1"])
# sample_key2 = sample_key2[x.col1]
select!(x,Not("col1"));

println(size(x))
println(size(y))
# println(length(sample_key2))
println(sum([i==true for i in y]));

46
(13108,)

(2530, 40)
(2530,)
5


In [48]:
transform_log!(x,10)
stnd = fit_standardizer(x)
transform_standardizer!(x,stnd);
println(sum(ismissing.(Matrix(x))))

1859


In [49]:
imputemach = machine(MissingImputator(K=5),x) 
@time fit!(imputemach,verbosity=1)

┌ Info: Training Machine{MissingImputator,…}.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/HZmTU/src/machines.jl:403


  2.091787 seconds (13.70 M allocations: 1.515 GiB, 14.00% gc time, 14.01% compilation time)


Machine{MissingImputator,…} trained 1 time; caches data
  args: 
    1:	Source @646 ⏎ `Table{Union{AbstractVector{Union{Missing, Continuous}}, AbstractVector{Continuous}}}`


In [50]:
tt = MLJ.transform(imputemach,x);
for i in 1:ncol(x)
    x[:,i] = tt[i]
end
disallowmissing!(x);

In [51]:
println(size(x)," ",sum([i==true for i in y] ) )
println(size(y))
# x,y,train,test,sample_key = sampler(x,y,train,test,sample_key,"under",0.8);
# x,y,train,test,sample_key = sampler(x,y,train,test,sample_key,"over",0.2);

y = coerce(copy(y),OrderedFactor);

println(size(x)," ",sum([i==true for i in y] ) )
println(size(y))

(2530, 40) 5
(2530,)
(2530, 40) 5
(2530,)


In [52]:

r = [
    range(EvoTreeClassifier(), :max_depth, lower=2,upper=8),
    range(EvoTreeClassifier(), :η, lower=0.0,upper=1.0),
    range(EvoTreeClassifier(), :γ, lower=0.0,upper=1.0),
    range(EvoTreeClassifier(), :λ, lower=0.0,upper=1.0),
    range(EvoTreeClassifier(), :α, lower=0.0,upper=1.0),
    range(EvoTreeClassifier(), :min_weight, lower=0.0,upper=1.0),
    range(EvoTreeClassifier(), :nrounds, lower=1,upper=10),
    range(EvoTreeClassifier(), :nbins, lower=4, upper=128),
    
    ]

8-element Vector{MLJBase.NumericRange{T, MLJBase.Bounded, Symbol} where T}:
 NumericRange(2 ≤ max_depth ≤ 8; origin=5.0, unit=3.0)
 NumericRange(0.0 ≤ η ≤ 1.0; origin=0.5, unit=0.5)
 NumericRange(0.0 ≤ γ ≤ 1.0; origin=0.5, unit=0.5)
 NumericRange(0.0 ≤ λ ≤ 1.0; origin=0.5, unit=0.5)
 NumericRange(0.0 ≤ α ≤ 1.0; origin=0.5, unit=0.5)
 NumericRange(0.0 ≤ min_weight ≤ 1.0; origin=0.5, unit=0.5)
 NumericRange(1 ≤ nrounds ≤ 10; origin=5.5, unit=4.5)
 NumericRange(4 ≤ nbins ≤ 128; origin=66.0, unit=62.0)

In [53]:
self_tune = TunedModel(
#     model=LogisticClassifier(penalty=:l1),
    model=EvoTreeClassifier(),
    
    resampling=StratifiedCV(nfolds=5, rng=123),

    tuning=LatinHypercube(),  #     tuning=Grid(resolution=2), 
    range = r,
    n=200,
    operation=predict_mode, # use for probabalistic 
    measure = fnr,  # use for deterministic or with predict_mode for probab 
    acceleration=CPUThreads(),
)

machbest = machine(
    self_tune,
    x,y)
fit!(machbest,verbosity=1 )

mach = machine(
    fitted_params(machbest).best_model,
    x,y)
fit!(mach,verbosity=1 )

┌ Info: Training Machine{ProbabilisticTunedModel{LatinHypercube,…},…}.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/HZmTU/src/machines.jl:403
┌ Info: Attempting to evaluate 200 models.
└ @ MLJTuning /Users/alexpanchot/.julia/packages/MLJTuning/efiDR/src/tuned_models.jl:685
Evaluating over 200 metamodels: 100%[=========================] Time: 0:00:21
┌ Info: Training Machine{EvoTreeClassifier{Float64,…},…}.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/HZmTU/src/machines.jl:403


Machine{EvoTreeClassifier{Float64,…},…} trained 1 time; caches data
  args: 
    1:	Source @223 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @318 ⏎ `AbstractVector{OrderedFactor{2}}`


In [54]:
FileIO.save("pindel$(full).jld2", Dict("mach"=>mach,"stnd" => stnd,
        "imputemach" => imputemach, 
    ) ; compress = true
)